# Planetary Motion
## 20 March 2023

In [21]:
# Imports
using Plots, LinearAlgebra
const G = 6.67430e-11  # Gravitational constant (m^3 kg^-1 s^-2)

6.6743e-11

# Physics background
Consider two gravitationally interacting planets:

<div> <img src="Planets.png" alt="Drawing" style="width: 400px;"/></div> 

The force on planet $i$ due to planet $j$ is given by
$$ \vec{F}_{ij} = \frac{G m_i m_j}{r_{ji}^2} \hat{r_{ij}}, $$
where 
$$\hat{r_{ij}} = \frac{\vec{r_{ij}}}{r_{ij}}$$ 
and $\vec{r_{ij}} = \vec{r}_j - \vec{r}_i$. The force on planet $j$ exerts on planet $i$ is, by Newton's third law, the same magnitude but in the opposite direction. 

Now suppose we want to simulate the motion of these two objects. Newton's 2nd Law gives us

$$ m_i\frac{d\vec{v_i}}{dt} =  -\frac{G m_i m_j}{r_{ij}^2} \hat{r_{ij}},$$

and 

$$ m_j \frac{d\vec{v_j}}{dt} =  -\frac{G m_i m_j}{r_{ji}^2} \hat{r_{ji}}.$$

Canceling masses, we are left with 

$$ \frac{d\vec{v_i}}{dt} =  -\frac{G  m_j}{r_{ij}^2} \hat{r_{ij}},$$

and 
$$ \frac{d\vec{v_j}}{dt} =  -\frac{G m_i }{r_{ji}^2} \hat{r_{ji}}.$$

Together with 

$$ \frac{d\vec{r_i}}{dt} = \vec{v_i}$$

$$ \frac{d\vec{r_j}}{dt} = \vec{v_j}$$

we have equations to simulate the system of two masses. 

# Simulation
Because we expect periodic motion, we will not use the Euler method, but rather the Euler-Cromer method.
Before jumping into writing this code, we will introduce a new feature of Julia: the *struct*. A *struct* is a custom type in Julia. The easiest way to see how this works is to define one. For this problem, I'll create a struct called Planet:

In [1]:
struct Planet
    m::Float64             # mass
    r::Vector{Float64}     # position vector
    v::Vector{Float64}     # velocity vector
end

The above code defines a new type (called Planet) with three attributes: *mass*, *position* and *velocity*. The types of each object are defined in the struct definition. 

Let's define the moon and the earth as two separate instances of Planet; I'll assume that the earth is at rest to start:

In [2]:
moon =  Planet(7.348e22, [3.84401e8, 0.0], [0.0, 1022]);
earth = Planet(5.972e24, [0.0, 0.0], [0.0, 0.0]);

Now suppose we want to find out the mass of the moon. We simply execute

In [3]:
moon.m

7.348e22

If you've done any object oriented programming, you'll see that this notation is identical. The velocity of the earth, for instance, is simply:

In [4]:
earth.v

2-element Vector{Float64}:
 0.0
 0.0

Where is the center of mass of the Earth-moon System? 

In [6]:
cm = (moon.m * moon.r .+ earth.m*earth.r)/(moon.m + earth.r)

LoadError: MethodError: no method matching +(::Float64, ::Vector{Float64})
For element-wise addition, use broadcasting with dot syntax: scalar .+ array

[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m)
[0m[90m   @[39m [90mBase[39m [90m[4moperators.jl:578[24m[39m
[0m  +(::T, [91m::T[39m) where T<:Union{Float16, Float32, Float64}
[0m[90m   @[39m [90mBase[39m [90m[4mfloat.jl:408[24m[39m
[0m  +(::Union{Float16, Float32, Float64}, [91m::BigFloat[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mmpfr.jl:423[24m[39m
[0m  ...


So, now let's define our system of two bodies:

In [13]:
bodies = [moon, earth]

2-element Vector{Planet}:
 Planet(7.348e22, [3.84401e8, 0.0], [0.0, 1022.0])
 Planet(5.972e24, [0.0, 0.0], [0.0, 0.0])

Also, let's define time step, $\Delta t$, the simulation time, *tmax*, and the number of steps in the simulation:

In [14]:
Δt = 3600.0 # one hour
tmax = 86400 * 90 # 90 days
num_steps = Int(tmax/Δt)

2160

We will need to compute the acceleration of each object, so let's make a function to do this:


In [19]:
function accel(bodyi::Planet, bodyj::Planet)
    rji = bodyj.r - bodyi.r
    r = norm(rji)
    aᵢ= - (G*bodyj.m/r^3)*rji
end

accel (generic function with 1 method)

Now let's write a function to implement the Euler-Cromer method:

In [20]:
function simulate(bodies, num_steps, Δt)
    num_bodies = length(bodies)
    positions = [zeros(num_steps, 2) for _ in 1:num_bodies]

    for i in 1:num_bodies    # initialize positions for all objects
        positions[i][1, :] = bodies[i].position
    end

    for step = 2:num_steps
        for i in 1:num_bodies
            a_total = zeros(2)
            for j in 1:num_bodies
                if i != j
                    a = accel(bodies[i], bodies[j])
                    a_total += a
                end
            end
            bodies[i].velocity =  bodies[i].velocity .+ a_total * Δt
            bodies[i].position = bodies[i].position .+bodies[i].velocity * Δt
            positions[i][step, :] .= bodies[i].position
        end
    end

    return positions
end
    

simulate (generic function with 1 method)

In [22]:
simulate(bodies, num_steps, Δt)


LoadError: type Planet has no field position